In [2]:
import numpy as np
import pandas as pd

pandas supports many datatypes...<br>
check the main jupyter page for more info 

pandas has to ways to store strings : 
- object
- StringDtype

In [14]:
df = pd.DataFrame(
    {
        "A": [1, 2, 3],
        "B": [1.2, 2.2, 0.4],
        "C": ["hello", "a", "B"],
        "D": pd.Series(pd.date_range("2002/1/1", periods=3,  freq="M")),
        "E": False,
        "F": pd.Series([1, 2, 3], dtype="int8"),
    }
)
df

,A,B,C,D,E,F
0,1,1.2,hello,2002-01-31,False,1
1,2,2.2,a,2002-02-28,False,2
2,3,0.4,B,2002-03-31,False,3


always check the dtype of your dataframe columns : 

columns A and F look the same, but if we check theri dtype, it shows that they are not completyly the same

In [15]:
df.dtypes

A             int64
B           float64
C            object
D    datetime64[ns]
E              bool
F              int8
dtype: object

the mumbers of columns of each type in a DataFrame can be found by calling `DataFrame.dtypes.value_counts()`

In [16]:
df.dtypes.value_counts()

int64             1
float64           1
object            1
datetime64[ns]    1
bool              1
int8              1
dtype: int64

### upcasting

In [17]:
df1 = pd.DataFrame({"A": np.random.randn(5)})
df1

,A
0,1.022202
1,-1.679857
2,-1.006968
3,1.473950
4,-0.321038


In [18]:
df1.dtypes

A    float64
dtype: object

In [19]:
df2 = pd.DataFrame(
    {
        "A": np.random.randn(5).astype('float32'),
        "B": np.random.randn(5),
        "C": np.random.randn(5).astype('uint8'),
    }
)
df2

,A,B,C
0,1.162296,-0.721798,0
1,-1.872885,0.276769,0
2,0.656854,-1.488564,0
3,-0.650797,1.090862,2
4,-0.121598,-0.313899,255


In [20]:
df2.dtypes

A    float32
B    float64
C      uint8
dtype: object

In [27]:
df3 = df1.reindex_like(df2).fillna(value=0.0) + df2
df3

,A,B,C
0,2.184499,-0.721798,0.0
1,-3.552743,0.276769,0.0
2,-0.350113,-1.488564,0.0
3,0.823153,1.090862,2.0
4,-0.442637,-0.313899,255.0


In [28]:
df3.dtypes

A    float64
B    float64
C    float64
dtype: object

it returns the dtype that can accommodate ALL of the types in the resulting homogeneous dtyped Numpy array.<br>

### astype

In [30]:
df1.dtypes

A    float64
dtype: object

In [31]:
df1.astype("float16").dtypes

A    float16
dtype: object

### object conversion

In [41]:
df = pd.DataFrame(
    [
        [1, 2], 
        ['a', 'b'],
        pd.Series(pd.date_range('2002-2-1', '2022-1-1',periods=2)),
    ]
)
df

,0,1
0,1,2
1,a,b
2,2002-02-01 00:00:00,2022-01-01 00:00:00


In [42]:
df.dtypes

0    object
1    object
dtype: object

In [44]:
df2 = df.T
df2

,0,1,2
0,1,a,2002-02-01
1,2,b,2022-01-01


In [45]:
df2.dtypes

0            object
1            object
2    datetime64[ns]
dtype: object

the data in column 1 and column 2 are stored as `object`, we can fix it by `infer_objects`

In [48]:
df2 = df2.infer_objects()
df2

,0,1,2
0,1,a,2002-02-01
1,2,b,2022-01-01


In [50]:
df2.dtypes

0             int64
1            object
2    datetime64[ns]
dtype: object

and now it is fiexed!<br><br>

#### `to_numeric`

In [52]:
m = [1, '1.2', 3.23]
pd.to_numeric(m)

array([1.  , 1.2 , 3.23])

#### `to_datetime`

In [54]:
import datetime

In [55]:
m = ['2016', datetime.datetime(2020, 1, 22), '2000/2/2']

In [56]:
pd.to_datetime(m)

DatetimeIndex(['2016-01-01', '2020-01-22', '2000-02-02'], dtype='datetime64[ns]', freq=None)

#### `to_timedelta`

In [57]:
m = ['5us', pd.Timedelta('1day')]

In [59]:
pd.to_timedelta(m)

TimedeltaIndex(['0 days 00:00:00.000005', '1 days 00:00:00'], dtype='timedelta64[ns]', freq=None)

to force conversion, we can pass in `errors` argument, which specifies how pandas should deal with elements that cannot be converted to desired dtype or object.<br>
by default, `erros='raise`, meaning that any erros encountered will be raised during the conversion process.<br>
`errors='coerce` --> the errors will be ignored and pandas will convert problematic elements to `pd.NaT` or `pd.nan`<br><br>

In [69]:
m = [1, 2.2, 'abc']

In [70]:
pd.to_numeric(m, errors='raise') # default

ValueError: Unable to parse string "abc" at position 2

In [67]:
pd.to_numeric(m, errors='coerce')

array([1. , 2.2, nan])

In [71]:
pd.to_numeric(m, errors='ignore')

array([1, 2.2, 'abc'], dtype=object)

### Selecting columns based on dtype

In [73]:
df = pd.DataFrame(
    {
        "string": list("abc"),
        "int64": list(range(1, 4)),
        "uint8": np.arange(3, 6).astype("u1"),
        "float64": np.arange(4.0, 7.0),
        "bool1": [True, False, True],
        "bool2": [False, True, False],
        "dates": pd.date_range("now", periods=3),
        "category": pd.Series(list("ABC")).astype("category"),
    }
)

,string,int64,uint8,float64,bool1,bool2,dates,category
0,a,1,3,4.0,True,False,2022-11-27 02:42:08.332650,A
1,b,2,4,5.0,False,True,2022-11-28 02:42:08.332650,B
2,c,3,5,6.0,True,False,2022-11-29 02:42:08.332650,C


In [75]:
df["uint64"] = np.arange(3, 6).astype("u8")
df["other_dates"] = pd.date_range("20130101", periods=3)
df["tz_aware_dates"] = pd.date_range("20130101", periods=3, tz="US/Eastern")

df

,string,int64,uint8,float64,bool1,bool2,dates,category,uint64,other_dates,tz_aware_dates
0,a,1,3,4.0,True,False,2022-11-27 02:42:08.332650,A,3,2013-01-01,2013-01-01 00:00:00-05:00
1,b,2,4,5.0,False,True,2022-11-28 02:42:08.332650,B,4,2013-01-02,2013-01-02 00:00:00-05:00
2,c,3,5,6.0,True,False,2022-11-29 02:42:08.332650,C,5,2013-01-03,2013-01-03 00:00:00-05:00


In [76]:
df.dtypes

string                                object
int64                                  int64
uint8                                  uint8
float64                              float64
bool1                                   bool
bool2                                   bool
dates                         datetime64[ns]
category                            category
uint64                                uint64
other_dates                   datetime64[ns]
tz_aware_dates    datetime64[ns, US/Eastern]
dtype: object

In [77]:
df.select_dtypes('uint8')

,uint8
0,3
1,4
2,5


In [78]:
df.select_dtypes('bool')

,bool1,bool2
0,True,False
1,False,True
2,True,False


In [80]:
df.select_dtypes(['bool', 'object'])

,string,bool1,bool2
0,a,True,False
1,b,False,True
2,c,True,False


In [82]:
df.select_dtypes(include=['bool', 'object']) # the same as above

,string,bool1,bool2
0,a,True,False
1,b,False,True
2,c,True,False


In [84]:
df.select_dtypes(exclude=['bool']) # everything but bool type

,string,int64,uint8,float64,dates,category,uint64,other_dates,tz_aware_dates
0,a,1,3,4.0,2022-11-27 02:42:08.332650,A,3,2013-01-01,2013-01-01 00:00:00-05:00
1,b,2,4,5.0,2022-11-28 02:42:08.332650,B,4,2013-01-02,2013-01-02 00:00:00-05:00
2,c,3,5,6.0,2022-11-29 02:42:08.332650,C,5,2013-01-03,2013-01-03 00:00:00-05:00
